In [42]:
import pandas as pd
import numpy as np

from tqdm import tqdm

import sys

from astropy.io import fits

In [66]:
data_folder = 'C:/Users/oryan/Documents/mergers-in-cosmos/back-to-basics-data'
folder = 'C:/Users/oryan/Documents/mergers-in-cosmos'
student_folder = 'C:/Users/oryan/Documents/mergers-in-cosmos/followup-data'
results_folder = 'C:/Users/oryan/Documents/mergers-in-cosmos/back-to-basics-results'
cosmos_folder = 'E:/cosmos-data'

### Getting Student Data

In [4]:
df = pd.read_csv(f'{student_folder}/group_cosmos_with_main_cosmos_matches_OUT1.csv', index_col = 0)

In [8]:
df_red = df[['SourceID','ra_1','dec_1']]

In [21]:
df

,SourceID,RA_1a,Dec_1a,isStageOne,isStageTwo,isStageThree,isStageFour,ra_1,dec_1,type_1,...,m_k_2,mass_med_2,mass_med_min68_2,mass_med_max68_2,mass_best_2,sfr_med_2,sfr_med_min68_2,sfr_med_max68_2,sfr_best_2,Separation_2
col1,,,,,,,,,,,,,,,,,,,,,
21,4000705532455,150.679664,2.196615,False,True,True,False,150.679866,2.196563,0,...,-19.907,8.457,8.268,8.648,8.127,0.321,0.215,0.557,0.789,2.568817
22,4000705532984,150.673546,2.226438,True,False,False,False,150.673687,2.226308,0,...,-18.858,9.149,9.029,9.229,9.218,-0.343,-0.504,-0.210,-0.359,4.852455
23,4000705533312,150.667975,2.242945,False,False,True,False,150.668120,2.242867,0,...,-23.430,10.750,10.716,10.784,10.763,0.500,0.432,0.569,0.505,2.646160
24,4000705533383,150.645125,2.237650,True,False,False,False,150.645145,2.237538,0,...,-20.918,9.461,9.411,9.508,9.480,1.013,0.924,1.091,1.032,2.257583
25,4000705539435,149.700729,2.634871,False,False,False,True,149.700540,2.634988,0,...,-18.228,8.084,7.835,8.328,8.054,-0.145,-0.249,0.176,-0.136,7.357721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21920,6000536185363,150.498697,2.651979,True,False,False,False,150.499015,2.652113,0,...,-22.949,10.900,10.866,10.934,10.889,-0.201,-0.269,-0.132,-0.231,3.121607
21921,6000536185496,150.500839,2.662972,True,False,False,False,150.501189,2.663116,0,...,-21.284,9.304,9.187,9.380,9.299,1.320,1.239,1.388,1.298,4.824996
21922,6000536185585,150.487245,2.671166,False,False,True,False,150.487626,2.671314,0,...,-23.149,10.277,10.179,10.376,10.273,1.695,1.337,2.040,1.753,1.779700


### Getting COSMOS Data

In [13]:
with fits.open(f'{cosmos_folder}/cosmos2015v21.fits.gz') as hdul:
    cosmos_data = hdul[1].data

### Matching Student Data to COSMOS ID

In [15]:
ra = df_red.ra_1.iloc[0]
dec = df_red.dec_1.iloc[0]

In [31]:
np.round(ra, 6)

150.679866

In [34]:
cosmos_data[(np.round(cosmos_data['ALPHA_J2000'],6) == ra) & (np.round(cosmos_data['DELTA_J2000'], 6) == dec)]

1

In [39]:
global cosmos_data

In [55]:
def get_id(ra, dec):
    row = cosmos_data[(np.round(cosmos_data['ALPHA_J2000'],6) == ra) & (np.round(cosmos_data['DELTA_J2000'], 6) == dec)]
    if len(row) > 1:
        print('Duplicate')
        sys.exit()
    idn = row['NUMBER'][0]
    return idn

In [59]:
# ids_dict = {}

In [60]:
done_ids = list(ids_dict.keys())
for i in tqdm(list(df_red.SourceID)):
    if i in done_ids:
        continue
    row = df_red.query('SourceID == @i').iloc[0]
    ids_dict[i] = get_id(row['ra_1'], row['dec_1'])

100%|██████████| 3803/3803 [28:04<00:00,  2.26it/s]


In [64]:
df_ids = pd.DataFrame.from_dict(ids_dict, orient = 'index').rename(columns = {0 : 'id_2015'})

In [67]:
df_ids.to_csv(f'{data_folder}/cosmos2015_ids.csv')